# Documentation assistant

This notebook demonstrates a documentation assistant: Video-to-documentation conversion using Vertex AI

Converting videos-to-documentation involves three steps: 
1. Protocol finder: Select protocol which best captures the step being performed in the video
2. Video comparing to ground-of-truth protocol → lab documentation + errors in procedure
3. Analytics based on benchmark dataset: automatic comparison of errors found by documentation assistent vs actual errors

In this notebook, I will focus on the first step - the protocol finder.

In [1]:
# %pip install google-cloud-storage
# %pip install --upgrade --user --quiet google-cloud-aiplatform

In [2]:
from __future__ import annotations

# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

import configparser
import os
import sys
from pathlib import Path

from IPython.display import Markdown

path_to_append = Path(Path.cwd()).parent / "proteomics_specialist"
sys.path.append(str(path_to_append))
import video_to_protocol

config = configparser.ConfigParser()
config.read("../secrets.ini")

['../secrets.ini']

In [3]:
import vertexai

config = configparser.ConfigParser()
config.read("../secrets.ini")

PROJECT_ID = config["DEFAULT"]["PROJECT_ID"]
vertexai.init(project=PROJECT_ID, location="europe-west9")  # europe-west9 is Paris

In [4]:
from google.cloud import storage

os.environ["GOOGLE_CLOUD_PROJECT"] = config["DEFAULT"]["PROJECT_ID"]

# Initialize Cloud Storage client
storage_client = storage.Client()
bucket_name = "mannlab_videos"
bucket = storage_client.bucket(bucket_name)

In [ ]:
from vertexai.generative_models import Part
from vertexai.preview.generative_models import GenerativeModel

MODEL_ID = "gemini-1.5-pro-001"

In [6]:
# Upload protocols to GCS

directory = "/Users/patriciaskowronek/Documents/proteomics_specialist/data/"

list_protocol_gcs = [
    video_to_protocol.upload_video_to_gcs(Path(directory) / file, bucket, "protocol_finder")
    for file in os.listdir(directory) 
    if "protocol" in file.lower()
]

list_protocol_gcs

['gs://mannlab_videos/protocol_finder/QueueSamples_protocolCorrect.md',
 'gs://mannlab_videos/protocol_finder/PlaceEvotips_protocolCorrect.md',
 'gs://mannlab_videos/protocol_finder/DisconnectColumn_protocolCorrect.md',
 'gs://mannlab_videos/protocol_finder/Diluting_protocolCorrect.md',
 'gs://mannlab_videos/protocol_finder/TimsCalibration_protocolCorrect.md',
 'gs://mannlab_videos/protocol_finder/ConnectingColumnSampleLine_protocolCorrect.md',
 'gs://mannlab_videos/protocol_finder/UltraSourceToESIsource_protocolCorrect.md',
 'gs://mannlab_videos/protocol_finder/Pipette_protocolCorrect.md',
 'gs://mannlab_videos/protocol_finder/ESIsourceToUltraSource_protocolCorrect_CapillaryPushedIn.md',
 'gs://mannlab_videos/protocol_finder/Evotip_protocolCorrect.md']

In [7]:
# Prepare protocols as model input

protocol_input = []
count = 1
for protocol in list_protocol_gcs:
    protocol_input.append("number: " + str(count))
    protocol_input.append("file name: " + os.path.basename(protocol))
    protocol_input.append(Part.from_uri(protocol, mime_type="text/md"))
    count += 1

In [8]:
# Prepare video for model input
video_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/PlaceEvotips_docuLiquidNotChecked.MP4"
video_uri = video_to_protocol.upload_video_to_gcs(video_path, bucket, "protocol_finder")

video_input = [
    "Lab video:",
    Part.from_uri(video_uri, mime_type="video/mp4"),
]

In [9]:
# Prompt

import enum

class ProtocolClass(enum.Enum):
    protocol1 = "1"
    protocol2 = "2"
    protocol3 = "3"
    protocol4 = "4"
    protocol5 = "5"

inputs = [
    """
    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry.
    Your task is to find the protocol that captures the step being performed in the video. 
    Protocols:
    """
]

inputs.extend(protocol_input)
inputs.extend(video_input)

inputs.append(
    "Output: Protocol number."
)

In [10]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, GenerationConfig

model = GenerativeModel("gemini-1.5-flash-002")

response = model.generate_content(
    inputs, 
    generation_config=GenerationConfig(
        temperature=0.9,
    )
    )

print(video_path)
verbose_eval = response.text
print(response.usage_metadata)
print(verbose_eval)


/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/PlaceEvotips_docuLiquidNotChecked.MP4
prompt_token_count: 17654
candidates_token_count: 25
total_token_count: 17679
prompt_tokens_details {
  modality: VIDEO
  token_count: 10545
}
prompt_tokens_details {
  modality: TEXT
  token_count: 7109
}
candidates_tokens_details {
  modality: TEXT
  token_count: 25
}

The video shows the step of spinning Evotips in a centrifuge. Therefore, protocol number 10 is the correct one.


In [11]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from IPython.display import Markdown

def identify_protocol_from_video(
    model_input,
    # options,
    model_name="gemini-1.5-flash-002", 
    temperature=0.9,
):
    """Identifies the protocol number from a lab video using Gemini model with customizable prompts.
    
    Parameters
    ----------
    model_input : object
        Input to be processed by the model
    model_name : str
        Name of the Gemini model to use
    temperature : float
        Temperature for model generation
    
    Returns
    -------
    dict
        Dictionary containing the model response and metadata including:
        - observation: The text response from the model
        - usage_metadata: Usage statistics from the model
        - markdown_observation: Markdown formatted observation
    """
    model = GenerativeModel(model_name) #, client_options=options)
    response = model.generate_content(
        model_input, 
        generation_config={"temperature": temperature}
    )
    verbose_eval = response.text
    
    return {
        "observation": verbose_eval,
        "usage_metadata": response.usage_metadata,
        "video_path": video_path,
        "markdown_observation": Markdown(verbose_eval)
    }

In [12]:
import enum

protocol_members = {}

for index, file_path in enumerate(list_protocol_gcs):
    member_name = os.path.basename(file_path)
    protocol_members[member_name] = str(index + 1)

# Dynamically create the enum class
ProtocolNum = enum.Enum('ProtocolNum', protocol_members)

In [13]:
def score_protocol_from_video(
    verbose_eval,
    protocol_enum=ProtocolNum,
    model_name="gemini-1.5-flash-002",
    temperature=0.9
):
    """Scores and classifies a protocol number from verbose evaluation using Gemini model.
    
    Parameters
    ----------
    verbose_eval : str
        The verbose evaluation text from a previous model run
    protocol_enum : Enum
        Enumeration containing valid protocol numbers
    model_name : str
        Name of the Gemini model to use
    temperature : float
        Temperature for model generation
        
    Returns
    -------
    dict
        Dictionary containing the model response and metadata including:
        - score_str: The raw text response from the model
        - structured_eval: The protocol number as enum value
        - usage_metadata: Usage statistics from the model
    """
    model = GenerativeModel(model_name)
    
    score_prompt = f"{verbose_eval}\n\nWhich protocol number is this? Please respond with only one of the following: {[e.value for e in protocol_enum]}."
    
    response_score = model.generate_content(
        score_prompt,
        generation_config=GenerationConfig(temperature=temperature, max_output_tokens=2)
    )
    
    score_str = response_score.text.strip()
    structured_eval = protocol_enum(score_str)
    
    return {
        "score_str": score_str,
        "structured_eval": structured_eval,
        "usage_metadata": response_score.usage_metadata
    }

In [14]:
# Evaluation if correct

import re
import os
import pandas as pd

def validate_protocol_number(video_path, csv_path, verbose_eval, ProtocolNum):
    """Validates if the protocol number in the observation matches the expected protocol number from the benchmark dataset (csv).
    
    Parameters
    ----------
    video_path : str
        Path to the video file
    csv_path : str
        Path to the benchmark dataset as csv file containing protocol information
    observation : str
        Text containing protocol number to be extracted
    
    Returns
    -------
    dict
        Result of the validation containing match status and relevant details including:
        - matches: Boolean indicating if protocol numbers match
        - video: Basename of the video file
        - expected_protocol: Protocol number from CSV
        - found_protocol: Protocol number extracted from observation
        - message: Optional explanation message (only present on failure)
    """
    video_basename = os.path.basename(video_path)
    
    df_link = pd.read_csv(csv_path, delimiter=";")
    df_link = df_link.dropna()
    
    if video_basename not in list(df_link["documentation video"]):
        return {
            "matches": False,
            "video": video_basename,
            "expected_protocol": [],
            "found_protocol": [],
            "message": f"Video not part of benchmark dataset: {video_basename}"
        }
    
    expected_protocol_number = str(df_link['Number'][df_link["documentation video"]==video_basename].item())

    try:
        result_score = score_protocol_from_video(
            verbose_eval,
            protocol_enum=ProtocolNum
        )
        found_protocol_number = result_score["score_str"]

        return {
            "matches": expected_protocol_number == found_protocol_number,
            "video": video_basename,
            "expected_protocol": expected_protocol_number,
            "found_protocol": found_protocol_number,
            "verbos_eval": verbose_eval
        }
    
    except (ValueError, KeyError, AttributeError, Exception) as e:
        # Catch any exception that might occur during scoring or protocol parsing
        return {
            "matches": False,
            "video": video_basename,
            "expected_protocol": expected_protocol_number,
            "found_protocol":f"Error extracting protocol number: {str(e)}",
            "verbose_eval": verbose_eval
        }
    

In [15]:
# First upload videos to Google Cloud Storage and convert to model input (reusable info, time consuming step)

video_directory = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation"

video_parts = {}

for video in os.listdir(video_directory):
    print(video)
    video_path = Path(video_directory) / video
    
    if video.lower().endswith(('.mp4', '.mov')):
        file_extension = os.path.splitext(video_path)[1].lower()[1:]
        
        video_uri = video_to_protocol.upload_video_to_gcs(video_path, bucket, "protocol_finder_2")

        video_parts[video] = {
            "path": str(video_path),
            "gcs_uri": video_uri,
            "part": [
                "Lab video:",
                Part.from_uri(video_uri, mime_type=f"video/{file_extension}")
            ]
        }

UltraSourceToESIsource_docuForgotN2Line.MP4
.DS_Store
TimsCalibration_docuCorrect_camera.MP4
TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4
PlaceEvotips_docuBoxAngeled.MP4
UltraSourceToESIsource_docuForgotGlovesANDCapillaryCap.MP4
QueueSamples_docuCorrect_1.mov
QueueSamples_docuNoBlankNoSampleIDWrongMSmethod.mov
UltraSourceToESIsource_docuCorrect.MP4
DisconnectingColumn_docuWithoutStandby_1.MP4
QueueSamples_docuDiffSampleNameAndEvotipNotAtPosition.mov
Pipette_ducoBufferBInsteadBufferA.MP4
ESIsourceToUltraSource_docuFogotOvenPowerSupply.MP4
Pipette_docu_WrongRowB_B10Twice_B11LeftOut.MP4
ConnectingColumnSampleLine_docuWithoutStandbyANDtimsControl.mp4
TimsCalibration_docuSavedMethod.mov
Dilute_docuWrongSolvent.MP4
TimsCalibration_docuWrongOrderSteps.mov
Dilute_docuWrongVolume_PipettTipNotChanged.MP4
DisconnectingColumn_docuCorrect.MP4
Pipette_docuFirstTwoWrongVolumneANDA2Twice.MP4
PlaceEvotips_docuWrongPosition.MP4
TimsCalibration_docuCorrect.mov
QueueSamples_docuWrongRow_S3A1Twice.mo

In [ ]:
system_prompt = ["""
    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry.
    Your task is to find the protocol that captures the step being performed in the video.
    """]
protocol_intro= ["Protocols:"]
output_format = ["Output: Explanation why this protocol fits and then number of protocol e.g. 'protocol number: 1'. You always decide for one protocol number."]

csv_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/Link_protocol_lab_video.csv"

validation_list = list()
for video in video_parts: 
    print(f"Processing video: {video}")
    video_path = video_parts[video]["path"]
    order = [system_prompt, video_parts[video]["part"], protocol_intro, protocol_input, output_format ]
    model_input = list()
    for input in order:
        model_input.extend(input)

    result = identify_protocol_from_video(
        model_input,
        model_name="gemini-2.0-flash", 
        temperature=0.9
    )

    # print(result["usage_metadata"])
    print(result["video_path"])
    display(result["markdown_observation"]) 

    result_validation = validate_protocol_number(video_path, csv_path, result["observation"], ProtocolNum)
    print(result_validation)
    validation_list.append(result_validation["matches"])

success_ratio = sum(validation_list) / len(validation_list) if len(validation_list) > 0 else 0
print(f"Success rate: {success_ratio:.2%}")

Processing video: UltraSourceToESIsource_docuForgotN2Line.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/UltraSourceToESIsource_docuForgotN2Line.MP4


Okay, Professor Mann here. Based on the lab video I've reviewed, the lab technician is performing procedures related to the ion source of a Bruker timsTOF mass spectrometer, specifically, they are changing or maintaining the ion source. After careful consideration of the various protocols, the actions in the video most closely align with the following:

The actions in the video most closely align with protocol number: 9.

**Reasoning:**

The video shows the technician:

1.  Removing parts of an ion source.
2.  Changing equipment on the timsTOF mass spectrometer.
3. Handling and replacing components within the timsTOF mass spectrometer
4. The technician is working on a region of the mass spectrometer that is consistent with the location of the ion source.


{'matches': False, 'video': 'UltraSourceToESIsource_docuForgotN2Line.MP4', 'expected_protocol': '7', 'found_protocol': '9', 'verbos_eval': "Okay, Professor Mann here. Based on the lab video I've reviewed, the lab technician is performing procedures related to the ion source of a Bruker timsTOF mass spectrometer, specifically, they are changing or maintaining the ion source. After careful consideration of the various protocols, the actions in the video most closely align with the following:\n\nThe actions in the video most closely align with protocol number: 9.\n\n**Reasoning:**\n\nThe video shows the technician:\n\n1.  Removing parts of an ion source.\n2.  Changing equipment on the timsTOF mass spectrometer.\n3. Handling and replacing components within the timsTOF mass spectrometer\n4. The technician is working on a region of the mass spectrometer that is consistent with the location of the ion source.\n"}
Processing video: TimsCalibration_docuCorrect_camera.MP4
/Users/patriciaskowrone

Okay, let's analyze the video to determine the protocol being performed.

**Video Analysis:**

The video shows a computer screen displaying software related to mass spectrometry. Specifically, it shows a chromatogram, instrument settings (MS Settings, TIMS Settings), and a list of reference compounds. Actions performed include selection of a reference list, setting of calibration parameters such as detection range and width, and then initiating a calibration. Moreover, it shows that a TIMS device is being calibrated as it shows the score in the tab "Calibration" is at 100%.

**Protocol Matching:**

Based on the video analysis, the activity being performed aligns with the procedure for calibrating the TIMS (Trapped Ion Mobility Spectrometer) device in a mass spectrometer.

**Therefore, I believe the correct protocol is:**

**protocol number: 5**


{'matches': True, 'video': 'TimsCalibration_docuCorrect_camera.MP4', 'expected_protocol': '5', 'found_protocol': '5', 'verbos_eval': 'Okay, let\'s analyze the video to determine the protocol being performed.\n\n**Video Analysis:**\n\nThe video shows a computer screen displaying software related to mass spectrometry. Specifically, it shows a chromatogram, instrument settings (MS Settings, TIMS Settings), and a list of reference compounds. Actions performed include selection of a reference list, setting of calibration parameters such as detection range and width, and then initiating a calibration. Moreover, it shows that a TIMS device is being calibrated as it shows the score in the tab "Calibration" is at 100%.\n\n**Protocol Matching:**\n\nBased on the video analysis, the activity being performed aligns with the procedure for calibrating the TIMS (Trapped Ion Mobility Spectrometer) device in a mass spectrometer.\n\n**Therefore, I believe the correct protocol is:**\n\n**protocol number: 

...

{'matches': False, 'video': 'UltraSourceToESIsource_docuForgotCapillaryCap.MP4', 'expected_protocol': '7', 'found_protocol': '9', 'verbos_eval': "Okay, let's analyze the video provided and determine the most appropriate protocol.\n\n**Analysis of the Video:**\n\nThe video shows a scientist working with a timsTOF mass spectrometer and associated equipment, including what appears to be an Evosep LC system. The key actions observed are:\n\n1.  **Disassembly/Assembly of parts within the UltraSource ion source:** The scientist removes components from within the UltraSource, including the spray shield. This involves some manipulation of small parts and a potential concern for cleanliness and correct reassembly. It appears the scientist either switches from an electrospray ionization source to another or changes the ion source within the UltraSource or performs maintenance work.\n2.  **General maintenance or cleaning** As the video continues, the scientist seems to be cleaning other parts and

Okay, let's analyze the video and determine the appropriate protocol.

The video shows a person in a lab, wearing gloves, who is placing Evotips (sample tips used with the Evosep system) into Evotip boxes on the Evosep One instrument. The person then proceeds to fill those tips with the sample.

Based on this action, the protocol that best describes the step being performed is **PlaceEvotips_protocolCorrect.md**. This protocol covers the inspection and placement of Evotips in Evotip boxes on the Evosep One system.

protocol number: 2

{'matches': True, 'video': 'PlaceEvotips_docuLiquidNotChecked.MP4', 'expected_protocol': '2', 'found_protocol': '2', 'verbos_eval': "Okay, let's analyze the video and determine the appropriate protocol.\n\nThe video shows a person in a lab, wearing gloves, who is placing Evotips (sample tips used with the Evosep system) into Evotip boxes on the Evosep One instrument. The person then proceeds to fill those tips with the sample.\n\nBased on this action, the protocol that best describes the step being performed is **PlaceEvotips_protocolCorrect.md**. This protocol covers the inspection and placement of Evotips in Evotip boxes on the Evosep One system.\n\nprotocol number: 2"}
Processing video: Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4


Okay, let's analyze the video. The person in the video is using a multichannel pipette to take tips and then spin them in a centrifuge. I think this best matches the 'Dispensing Protocol' protocol.

protocol number: 10

{'matches': True, 'video': 'Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4', 'expected_protocol': '10', 'found_protocol': '10', 'verbos_eval': "Okay, let's analyze the video. The person in the video is using a multichannel pipette to take tips and then spin them in a centrifuge. I think this best matches the 'Dispensing Protocol' protocol.\n\nprotocol number: 10"}
Success rate: 82.76%


In [37]:
# indices_to_select = [0, 4, 7, 26]

# # Get keys at specific positions
# all_keys = list(video_parts.keys())
# selected_keys = [all_keys[i] for i in indices_to_select if i < len(all_keys)]

# # Create dictionary with selected entries
# selected_videos = {k: video_parts[k] for k in selected_keys}

system_prompt = ["""
    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry with extensive laboratory experience.
    
    ## Your Background Knowledge:
    [These documents are for building your proteomics background knowldge and are not part of today's task.]
    """]

system_prompt_1 = [
    "## Today's Task:",
    "You need to analyze a laboratory video and identify which of the numbered protocols best matches the procedure being performed in the video.",
    """Your analysis must include these verification steps:
    1. Identify the starting state (describe visible features)
    2. List the specific actions taken in sequence while naming the involved equipment
    3. Identify the ending state (describe visible features)
    4. ONLY THEN match to a protocol number
    Your scientific reputation was built on exactitude - you cannot help but insist on proper technical terminology and chronological precision in all laboratory documentation."""
    ]

protocol_intro= ["## Available Protocols (You must select ONE):"]
output_format = ["""## Your response should be: Your explanation followed by "Protocol number: X" 
""",
f"Important: Choose exactly one protocol number. It is very important to you to focus only on matching the video to the protocols provided: {[e.value for e in ProtocolNum]}."
]

# Upload knowledge files to Google Cloud Storage
folder_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/knowledge_base_selected"
subfolder_in_bucket = "knowledge"

knowledge_uris = []
for filename in os.listdir(folder_path):
    if filename.lower().endswith(
        (".jpg", ".jpeg", ".gif", ".bmp", ".tiff", ".tif", ".pdf")
    ):
        path = Path(folder_path) / filename
        try:
            file_uri = video_to_protocol.upload_video_to_gcs(
                path, bucket, subfolder_in_bucket
            )
            knowledge_uris.append(file_uri)
        except OSError as e:
            print(f"Error processing {filename}: {e}")
MIME_TYPES = {
    ".pdf": "application/pdf",
    ".jpg": "image/jpeg",
    ".jpeg": "image/jpeg",
    ".png": "image/png",
}
contents = []
for file_path in knowledge_uris:
        path_obj = Path(file_path)
        file_ext = path_obj.suffix.lower()

        if file_ext in MIME_TYPES:
            mime_type = MIME_TYPES[file_ext]
            contents.append(Part.from_uri(file_path, mime_type=mime_type))

csv_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/Link_protocol_lab_video.csv"

validation_list = list()
for video in video_parts: 
    print(f"Processing video: {video}")
    video_path = video_parts[video]["path"]
    order = [system_prompt, contents, system_prompt_1, video_parts[video]["part"], protocol_intro, protocol_input, output_format]
    model_input = list()
    for input in order:
        model_input.extend(input)

    result = identify_protocol_from_video(
        model_input,
        model_name="gemini-2.0-flash", 
        temperature=0.9
    )

    # print(result["usage_metadata"])
    print(result["video_path"])
    display(result["markdown_observation"]) 

    result_validation = validate_protocol_number(video_path, csv_path, result["observation"], ProtocolNum)
    print(result_validation)
    validation_list.append(result_validation["matches"])

success_ratio = sum(validation_list) / len(validation_list) if len(validation_list) > 0 else 0
print(f"Success rate: {success_ratio:.2%}")

Processing video: UltraSourceToESIsource_docuForgotN2Line.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/UltraSourceToESIsource_docuForgotN2Line.MP4


Alright, let's meticulously analyze this video to determine the most suitable protocol.

1. **Starting State (0:00-0:01):**
   The video begins focused on the UltraSource of a timsTOF mass spectrometer. The column oven is open, revealing the column connection to what appears to be the Evosep sample line. A grounding screw is present. There is a black NanoViper adapter present.

2. **Observed Actions (0:01-2:31):**
   a. The technician moves to a computer and appears to interact with the TimsControl software, potentially related to source settings (0:11-0:17).
   b. The technician returns to the mass spectrometer (0:23) and loosens something at the UltraSource, detaching the grounding screw (0:24-0:29).
   c. The technician carefully disconnects the Evosep sample line from the column (0:32-0:35).
   d. The technician removes the column oven (0:40-0:47).
   e. The technician sets down the column oven and removes the UltraSource (0:52-0:58).
   f. The technician puts on gloves (1:01-1:07).
   g. The technician picks up the UltraSource door and attaches it to the device (1:10-1:20).
   h. The technician proceeds to place, arrange, and connect items to the top of the instrument (1:48-1:53)
   i. The technician moves to withdraw liquid with a syringe (1:55-2:07).
   j. The technician fills the sample line and connects the syringe to the system (2:10-2:31).

3. **Ending State (2:31-3:04):**
   The UltraSource is removed and the ESI source is placed. The system is running and being monitored via computer interface (2:35-3:04)

Given these observations, the procedure most closely aligns with the swapping of the ion source from the UltraSource to the ESI source. Crucially, the disconnection of the IonOpticks column at the beginning, and the connection of the source door, as well as the connection of the syringe at the end, confirm that the transition from the UltraSource to the ESI source is what has occurred.

Protocol number: 7

{'matches': True, 'video': 'UltraSourceToESIsource_docuForgotN2Line.MP4', 'expected_protocol': '7', 'found_protocol': '7', 'verbos_eval': "Alright, let's meticulously analyze this video to determine the most suitable protocol.\n\n1. **Starting State (0:00-0:01):**\n   The video begins focused on the UltraSource of a timsTOF mass spectrometer. The column oven is open, revealing the column connection to what appears to be the Evosep sample line. A grounding screw is present. There is a black NanoViper adapter present.\n\n2. **Observed Actions (0:01-2:31):**\n   a. The technician moves to a computer and appears to interact with the TimsControl software, potentially related to source settings (0:11-0:17).\n   b. The technician returns to the mass spectrometer (0:23) and loosens something at the UltraSource, detaching the grounding screw (0:24-0:29).\n   c. The technician carefully disconnects the Evosep sample line from the column (0:32-0:35).\n   d. The technician removes the column oven 

Alright, let's analyze this video with the precision required for accurate laboratory documentation.

**1. Starting State:**

The video begins with a computer monitor displaying the timsControl software interface. Key features visible include:

*   A chromatogram view showing signal intensities over time, suggesting an active LC-MS run.
*   The instrument status indicates "Operating" and "Mobility OK", suggesting that the mass spectrometer is actively acquiring data with the TIMS device enabled.
*   Various settings are displayed on the left-hand side of the screen, related to MS settings and TIMS settings. These include parameters like "Scan Begin", "Scan End", "Ion Polarity", "Mode", "Ramp Time," and "Duty Cycle."
*   A table is visible, possibly displaying reference lists or calibration data.

**2. Actions Taken (Sequential):**

The video shows the following sequence of actions performed via the timsControl software interface:

1.  **Data Review/Analysis:** The focus remains on the computer screen. Throughout the video, the chromatogram is displayed and updated, suggesting the instrument is still running.
2.  **Parameter adjustments:** The user adjusts parameters in the software. For instance, "MS Averaging" is changed. The user adjusts settings related to calibration, focusing on ion mobility.
3. **Calibration Menu Navigation:** The user navigates to the 'Calibration' menu within timsControl.
4. **Calibration Execution:** The user clicks 'Calibrate', suggesting an attempt to re-calibrate the instrument based on the displayed data and parameters.
5. **Saving attempt:** The user attempts to save the changes and presses "Discard all changes"

**3. Ending State:**

The ending state mirrors the initial state, with the timsControl software interface displaying updated chromatogram data and modified calibration settings. The instrument status continues to indicate "Operating" and "Mobility OK"

**4. Protocol Matching:**

Based on the observed actions, the protocol that best matches the video is **Protocol 5: Calibrate the TIMS device**. The video clearly shows attempts to adjust calibration parameters and navigate the timsControl software to initiate a mobility calibration, which aligns with the key procedures outlined in Protocol 5.

Protocol number: 5


{'matches': True, 'video': 'TimsCalibration_docuCorrect_camera.MP4', 'expected_protocol': '5', 'found_protocol': '5', 'verbos_eval': 'Alright, let\'s analyze this video with the precision required for accurate laboratory documentation.\n\n**1. Starting State:**\n\nThe video begins with a computer monitor displaying the timsControl software interface. Key features visible include:\n\n*   A chromatogram view showing signal intensities over time, suggesting an active LC-MS run.\n*   The instrument status indicates "Operating" and "Mobility OK", suggesting that the mass spectrometer is actively acquiring data with the TIMS device enabled.\n*   Various settings are displayed on the left-hand side of the screen, related to MS settings and TIMS settings. These include parameters like "Scan Begin", "Scan End", "Ion Polarity", "Mode", "Ramp Time," and "Duty Cycle."\n*   A table is visible, possibly displaying reference lists or calibration data.\n\n**2. Actions Taken (Sequential):**\n\nThe vide

...

{'matches': False, 'video': 'UltraSourceToESIsource_docuForgotCapillaryCap.MP4', 'expected_protocol': '7', 'found_protocol': '9', 'verbos_eval': "Alright, let's analyze this laboratory video with the precision it demands.\n\n1.  **Starting State:**\n    *   The timsTOF mass spectrometer is present, equipped with what appears to be a half-spherical housing indicative of an ESIsource.\n    *   The IonOpticks column is presumably disconnected, as the UltraSource, which is part of the Evosep system is not present. A corrugated sample line and other apparatus are visible nearby.\n    *   The instrument is presumably in standby mode.\n\n2.  **Actions Taken (in Sequence):**\n    *   0:24 The technician retrieves a section of tubing and a fitting from the desktop.\n    *   0:32 The technician carefully installs the tubing and fitting onto the inlet of the ESIsource.\n    *   0:50 The technician removes the half-spherical housing revealing a metal mounting bracket.\n    *   0:55 The technician 

Alright, let's analyze this video with the precision it deserves.

1.  **Starting State:** The video begins with a view of the Evosep One liquid chromatography system. The system's deck appears to be empty except for some accessories.

2.  **Observed Actions (Chronological Order):**

    *   The operator, wearing black gloves, places two Evotip boxes onto the designated positions on the Evosep One system. These boxes appear to be the 96-well format.
    *   The operator is seen filling the Evotip boxes with pipette tips (Evotips). The tips are placed in a row-by-row fashion from an organizer.

3.  **Ending State:** Two Evotip boxes are now positioned on the Evosep One system.  Both boxes contain several pipette tips in the wells.

4.  **Protocol Matching:**

Based on these observations, the procedure closely matches the description in "Placing Evotips in Evotip Boxes on the Evosep One System" protocol.

Protocol number: 2

{'matches': True, 'video': 'PlaceEvotips_docuLiquidNotChecked.MP4', 'expected_protocol': '2', 'found_protocol': '2', 'verbos_eval': 'Alright, let\'s analyze this video with the precision it deserves.\n\n1.  **Starting State:** The video begins with a view of the Evosep One liquid chromatography system. The system\'s deck appears to be empty except for some accessories.\n\n2.  **Observed Actions (Chronological Order):**\n\n    *   The operator, wearing black gloves, places two Evotip boxes onto the designated positions on the Evosep One system. These boxes appear to be the 96-well format.\n    *   The operator is seen filling the Evotip boxes with pipette tips (Evotips). The tips are placed in a row-by-row fashion from an organizer.\n\n3.  **Ending State:** Two Evotip boxes are now positioned on the Evosep One system.  Both boxes contain several pipette tips in the wells.\n\n4.  **Protocol Matching:**\n\nBased on these observations, the procedure closely matches the description in "Plac

Alright, let's analyze this video with the precision it deserves.

**1. Initial State:**
The scene presents a laboratory bench with various items arranged on it. These items include a rack holding multiple test tubes, bottles, boxes containing what appear to be pipette tips (Evotips), an orange tube rack, and a multichannel pipette. A white piece of paper with handwritten notes is also visible. Several boxes labelled "Evotip" are present.

**2. Actions Taken:**
The technician, wearing black gloves, picks up a multichannel pipette. Then, they proceed to firmly attach multiple Evotips to the pipette. Next, using the pipette, the technician seemingly picks up multiple Evotips from an Evotip box. The technician then picks up two boxes labelled 'Evotip' and 'Not low retention'. Then the technician walks away.
The technician returns with the boxes and places them into a centrifuge. The lid is closed and the centrifuge settings are adjusted (presumably to 800g for 1 minute), and finally the centrifuge is started.

**3. Ending State:**
The centrifuge is running, which contains Evotips. The scientist walks away and will presumably will return to remove the boxes containing Evotips from the centrifuge.

**4. Protocol Matching:**
Based on the observed actions, specifically using a multi channel pipette to pick up Evotips, placing the Evotips in a centrifuge, and setting the centrifuge parameters to spin them, the most appropriate matching protocol is:
Protocol number: 10

{'matches': True, 'video': 'Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4', 'expected_protocol': '10', 'found_protocol': '10', 'verbos_eval': 'Alright, let\'s analyze this video with the precision it deserves.\n\n**1. Initial State:**\nThe scene presents a laboratory bench with various items arranged on it. These items include a rack holding multiple test tubes, bottles, boxes containing what appear to be pipette tips (Evotips), an orange tube rack, and a multichannel pipette. A white piece of paper with handwritten notes is also visible. Several boxes labelled "Evotip" are present.\n\n**2. Actions Taken:**\nThe technician, wearing black gloves, picks up a multichannel pipette. Then, they proceed to firmly attach multiple Evotips to the pipette. Next, using the pipette, the technician seemingly picks up multiple Evotips from an Evotip box. The technician then picks up two boxes labelled \'Evotip\' and \'Not low retention\'. Then the technician walks away.\nThe technician returns with 

In [25]:
indices_to_select = [0, 4, 7, 8, 18, 26]

# Get keys at specific positions
all_keys = list(video_parts.keys())
selected_keys = [all_keys[i] for i in indices_to_select if i < len(all_keys)]
selected_keys

['UltraSourceToESIsource_docuForgotN2Line.MP4',
 'UltraSourceToESIsource_docuForgotGlovesANDCapillaryCap.MP4',
 'UltraSourceToESIsource_docuCorrect.MP4',
 'DisconnectingColumn_docuWithoutStandby_1.MP4',
 'DisconnectingColumn_docuCorrect.MP4',
 'UltraSourceToESIsource_docuForgotCapillaryCap.MP4']

In [36]:
model_input

["\n    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry with extensive laboratory experience.\n    \n    ## Your Background Knowledge:\n    [This is a reference library containing numerous technical documents about proteomics equipment and procedures. These are for your reference only and are not part of today's task.]\n    ",
 file_data {
   mime_type: "application/pdf"
   file_uri: "gs://mannlab_videos/knowledge/Connecting_or_disconnecting_column_2.pdf"
 },
 "## Today's Task:",
 'You need to analyze a laboratory video and identify which of the numbered protocols best matches the procedure being performed in the video.',
 'Your analysis must include these verification steps:\n    1. Identify the starting state (describe visible features)\n    2. List the specific actions taken in sequence while naming the involved equipment\n    3. Identify the ending state (describe visible features)\n    4. ONLY THEN match to a protocol number\n    It is v